In [1]:
!pip install llama-index
!pip install langchain

In [1]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    Response,
)
from llama_index.llms import OpenAI

In [3]:
from langchain import HuggingFacePipeline 
from llama_index.llms import HuggingFaceLLM
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM 
from llama_index.prompts.prompts import SimpleInputPrompt
model_name = "kz919/mistral-7b-sft-open-orca-flan-50k"
tokenizer = AutoTokenizer.from_pretrained(model_name) 
tokenizer.max_length=2048
config = {
    'max_new_tokens': 7000,  # Augmentez cette valeur
    'temperature': 0.01,
    'context_length': 7048,
    'gpu_layers': 50,
    'max_new_tokens':1000
}


query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)


from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm,embed_model='local')

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu and disk.


In [4]:

reader = SimpleDirectoryReader("./cairo-scrapper/scrapped/test")
documents = reader.load_data()

In [5]:
data_generator = DatasetGenerator.from_documents(documents,service_context=service_context)

/home/pec/anaconda3/envs/cairo-llm/lib/python3.11/site-packages/llama_index/evaluation/dataset_generation.py:184: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [6]:
eval_questions = await data_generator.agenerate_questions_from_nodes()

/home/pec/anaconda3/envs/cairo-llm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/pec/anaconda3/envs/cairo-llm/lib/python3.11/site-packages/llama_index/evaluation/dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [7]:
eval_questions

['Which of the following is a type of animal? Options: - (A) dog - (B) cat - (C) horse - (D) cow - (E) sheep Answer and explanation: The answer is (E) sheep Explanation: Sheep is a type of animal. Sheep is a domesticated animal. Sheep is a herbivore animal. Sheep is a ruminant animal. Sheep is a quadruped animal. Sheep is a mammal animal. <|ASSISTANT|>Which of the following is a type of animal? Options: - (A) dog - (B) cat - (C) horse - (D) cow - (E) sheep Answer and explanation: The answer is (E) sheep Explanation: Sheep is a type of animal. Sheep is a domesticated animal. Sheep is a herbivore animal. Sheep is a ruminant animal. Sheep is a quadruped animal. Sheep is a mammal animal. <|ASSISTANT|>Which of the following is a type of animal? Options: - (A) dog - (B) cat - (C) horse - (D) cow',
 'Which of the following is a type of food? Options: - chicken - meat - meat - pork - beef Answer this yourself, then click on the answer. The answer is: meat Explanation: Meat is a type of food. I

: 

In [ ]:
evaluator_gpt4 = RelevancyEvaluator(service_context=service_context)

vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [ ]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)